In [1]:
import numpy as np
import json, os
from nltk.corpus import wordnet as wn
from math import log
import glob
import tqdm
import json
import spacy
import matplotlib.pyplot as plt

In [2]:
def _getTextFile(langual):
    file_list = glob.glob(f'../data/stopwords/stopwords/*_{langual}.txt')
    files = ",".join(file_list)
    return files

def cleanText(english_txt):
    try:
        word_tokens = english_txt.split()
        filtered_word = [w for w in word_tokens if w not in stop_words and not w.isdigit()]
        filtered_word = [w + " " for w in filtered_word]
        return "".join(filtered_word)
    except:
        return np.nan

def detectLang(txt):
    try:
        return detect(txt)
    except:
        return np.nan

def cleanNonEnglish(txt):
    txt = re.sub(r'\W+', ' ', txt)
    txt = txt.lower()
    txt = txt.replace("[^a-zA-Z]", " ")
    word_tokens = txt.split()
    filtered_word = [w for w in word_tokens if all(ord(c) < 128 for c in w)]
    filtered_word = [w + " " for w in filtered_word]
    return "".join(filtered_word)

def countMinMaxAver(lines):
    min_len = 10000
    aver_len = 0
    max_len = 0
    for temp in lines:
        aver_len = aver_len + len(temp)
        if len(temp) < min_len:
            min_len = len(temp)
        if len(temp) > max_len:
            max_len = len(temp)
    aver_len = 1.0 * aver_len / len(lines)
    print('min_len : ' + str(min_len))
    print('max_len : ' + str(max_len))
    print('average_len : ' + str(aver_len))
    
def titleProcessing(title):
    #title = re.findall(r"title=\"(.*)\">", title)
    #title = title[0].replace(" ", "_")
    return re.findall(r"title=\"(.*)\">", title)

def contentProcessing(content, title_list_i):
    if content[0].strip() == title_list_i[0]:
        return title_list_i, content[1].strip()

In [3]:
stop_words=set()
for file in _getTextFile("en").split(","):
    for word in open(file):
        stop_words.add(word.strip())

In [4]:
sememe_entity_sence = np.load("../data/sememe_entity_sence.npy", allow_pickle=True).tolist()
title_content_lemmatization_sence = np.load("../data/title_content_lemmatization_sence.npy", allow_pickle=True).tolist()

In [6]:
title_list = list(title_content_lemmatization_sence.keys())

In [ ]:
stop_words.add("")
stop_words.add("'s")
stop_words.add("States")
stop_words.add("United")

In [9]:
cdv = np.load("./sememe_network_cdv_en_wordnet_2000.npy", allow_pickle=True).tolist()

In [7]:
tagme_sememe_dict_l = {}
for items in tqdm.tqdm(sememe_entity_sence):
    if not items:
        continue 
    i, entityList = items.split("\t")
    if entityList == "null" or len(entityList) == 0:
        continue
    entityList = json.loads(entityList)
    entities = [d['spot'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
    #entities = [d['title'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
    title = title_list[int(i)]
    if title not in tagme_sememe_dict_l.keys():
        tagme_sememe_dict_l[title] = []
    entities = " ".join(entities).strip().split(" ")
    tagme_sememe_dict_l[title] = entities

100%|████████████████████████████████| 960853/960853 [00:24<00:00, 39763.21it/s]


In [10]:
len(tagme_sememe_dict_l)

960676

In [11]:
title_sememe_raw = {}
for k, v in tqdm.tqdm(tagme_sememe_dict_l.items()):
    sememe = []
    for w in v:
        if w in cdv:
        #if w in dict_sememes:
            sememe.append(w)
    if len(sememe) == 0:
        continue
    title_sememe_raw[k] = sememe

print("take max [:3000] in content lexion for sememe (raw)")
countMinMaxAver(title_sememe_raw)

100%|█████████████████████████████████| 960676/960676 [02:29<00:00, 6416.53it/s]


take max [:3000] in content lexion for sememe (raw)
min_len : 6
max_len : 216
average_len : 16.276665569937485


In [12]:
sememe_freq = {}
sememe_set = set()
for doc_sememes in title_sememe_raw.values():
    for sememe in doc_sememes:
        sememe_set.add(sememe)
        if sememe in sememe_freq:
            sememe_freq[sememe] += 1
        else:
            sememe_freq[sememe] = 1

sememe_lexion = list(sememe_set)
sememe_lexion_size = len(sememe_lexion)

sememe_id_map = {}
for i in range(sememe_lexion_size):
    sememe_id_map[sememe_lexion[i]] = i

print("sememe lexion size: ", sememe_lexion_size)

sememe lexion size:  1811


In [13]:
doc_word_freq = {}
for doc_id,(_, doc_words) in enumerate(title_sememe_raw.items()):
    for word in doc_words:
        word_id = sememe_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1
            
word_doc_list = {}
for i,(_, doc_words) in enumerate(title_sememe_raw.items()):
    appeared = set()
    for word in doc_words:
        if word in appeared:
            continue
        if word in word_doc_list:
            doc_list = word_doc_list[word]
            doc_list.append(i)
            word_doc_list[word] = doc_list
        else:
            word_doc_list[word] = [i]
        appeared.add(word)
        
word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)
    
tfidf_word2doc_all = {}
for i,(title, doc_words) in enumerate(title_sememe_raw.items()):
    doc_word_set = set()
    for word in doc_words:
        if word in doc_word_set or word == title:
            continue
        j = sememe_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        idf = log(1.0 * len(title_sememe_raw) / word_doc_freq[sememe_lexion[j]])
        tfidf_word2doc_all[key] = freq * idf
        doc_word_set.add(word)

In [58]:
tfidf_word2doc_all

{'0,225': 5.256696245845783,
 '0,167': 4.3875371026861725,
 '0,99': 11.52504103908429,
 '0,789': 7.1125108719870616,
 '0,1664': 4.164100768468392,
 '0,437': 6.035317636602908,
 '0,779': 7.937717774560073,
 '1,1259': 11.407410946463273,
 '1,79': 6.207277612831132,
 '1,430': 3.806165584917373,
 '1,1746': 5.605002940113998,
 '1,1244': 13.736570550001481,
 '1,1410': 5.851923902699566,
 '1,900': 7.278472145675809,
 '1,466': 5.782768680372092,
 '1,1672': 6.161105121092004,
 '2,1631': 25.031504886144013,
 '2,1023': 5.1409147941996975,
 '2,1013': 14.788963813286657,
 '2,430': 3.806165584917373,
 '2,570': 8.849887884275436,
 '2,1697': 4.973455933169944,
 '2,1726': 6.0968569539809865,
 '2,1798': 7.183715318278853,
 '2,1618': 7.014393016028539,
 '2,490': 5.541963830203046,
 '2,1728': 6.534894111488972,
 '3,12': 7.347754830235359,
 '3,965': 6.83259080487077,
 '3,1004': 6.844347961856814,
 '3,1052': 6.922202963560457,
 '4,712': 4.715865990098712,
 '4,1717': 5.166600722931748,
 '4,1222': 4.100840987

In [48]:
title_list[65]

'mbk (scooter manufacturer)>>>VBP'

In [49]:
sememe_lexion[1204]

'poem'

In [64]:
title_list = [word for word in title_sememe_raw.keys()]
tfidf_filter = {}
for title, tfidf in tfidf_word2doc_all.items():
    if tfidf > 4:
        w, d = title.split(",")
        if title_list[int(w)] not in tfidf_filter.keys():
            tfidf_filter[title_list[int(w)]] = []
        tfidf_filter[title_list[int(w)]].append(sememe_lexion[int(d)])
print("all lexion size:", len(tfidf_filter))

all lexion size: 600034


In [78]:
# this part for add pmi weight to building graph
title_list = [word for word in title_sememe_raw.keys()]
tfidf_filter = {}
for title, tfidf in tfidf_word2doc_all.items():
    if tfidf > 4:
        w, d = title.split(",")
        if title_list[int(w)] not in tfidf_filter.keys():
            tfidf_filter[title_list[int(w)]] = []
        pmi_weight = sememe_lexion[int(d)]+":"+str(tfidf)
        tfidf_filter[title_list[int(w)]].append(pmi_weight)
print("all lexion size:", len(tfidf_filter))

all lexion size: 600034


In [81]:
tfidf_filter = {k:v for k, v in tfidf_filter.items() if k in sememe_network.keys()}

In [82]:
print("all lexion size:", len(tfidf_filter))

all lexion size: 423249


In [83]:
np.save("wiki_sememe_423249_212", tfidf_filter)

In [65]:
title_list = list(title_content_lemmatization_sence.keys())

In [66]:
tagme_sememe_dict = {}
for items in tqdm.tqdm(sememe_entity_sence):
    if not items:
        continue 
    i, entityList = items.split("\t")
    if entityList == "null" or len(entityList) == 0:
        continue
    entityList = json.loads(entityList)
    #entities = [d['spot'] for d in entityList if 'title' in d and float(d['rho']) > 0.1]
    entities = [d['title']+"#"+str(d['link_probability']) for d in entityList if 'title' in d and float(d['rho']) > 0.01]
    #print(i)
    title = title_list[int(i)]
    if title not in tagme_sememe_dict.keys():
        tagme_sememe_dict[title] = []
    tagme_sememe_dict[title] = entities

100%|████████████████████████████████| 960853/960853 [00:28<00:00, 34229.03it/s]


In [67]:
title_list = list(title_content_lemmatization_sence.keys())
max_value = 4
sememe_network = {}
for word, sememes in tfidf_filter.items():
    if len(sememes) != 0:
        if len(sememes) > max_value:
            tagme_sememe_dict_v = tagme_sememe_dict[word]
            sememe_freq_max_value = {}
            for sememe in sememes:
                for v in tagme_sememe_dict_v:
                    #print(v)
                    s, l = v.split("#")
                    if s == sememe:
                        sememe_freq_max_value[sememe] = float(l)
            sememe_lower = sorted(sememe_freq_max_value.items(),key=lambda item:item[1],reverse=True)[:max_value]
            sememe_lower = [sememe_set[0] for sememe_set in sememe_lower]
            sememe_network[word] = sememe_lower
        else:
            sememe_network[word] = sememes
    else:
        continue

In [72]:
print("all lexion size:", len(sememe_network))

all lexion size: 423249


In [73]:
sum([len(v) for v in sememe_network.values()])/len(sememe_network)

2.1226677440466486

In [ ]:
wiki 423249 2.1226677440466486

In [74]:
word_freq = {}
word_set = set()
for words in sememe_network.values():
    for word in words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)

word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

print("vocab_size: ", vocab_size)

vocab_size:  1807


In [71]:
sememe_network = {k:v for k, v in sememe_network.items() if len(v) != 0}

In [37]:
len(list(sememe_network.values())[0])

0

In [75]:
sememe_network

{'deconfinement>>>JJ': ['physic', 'phase', 'matter', 'particle'],
 'horovitz>>>NN': ['middle', 'age', 'republic'],
 'cargolifter>>>NN': ['service', 'point', 'development', 'vessel'],
 'sk8er>>>NN': ['word', 'slang'],
 'qsound>>>NN': ['sound', 'processing'],
 'jumilla>>>NN': ['town'],
 "ta'anit>>>NN": ['classical', 'fast', 'food', 'drink'],
 'homoserine>>>NN': ['chemical', 'amino', 'acid', 'product'],
 'pc/sc>>>NN': ['card'],
 "strawbridge's>>>NN": ['store', 'northeastern', 'center'],
 'derivative (chemistry)>>>JJ': ['chemistry',
  'compound',
  'chemical',
  'reaction'],
 'vassarette>>>NN': ['division'],
 'diadumenos>>>NNS': ['sculptor', 'form'],
 'paisa (region)>>>VBP': ['department'],
 'pakan (electoral district)>>>JJ': ['single',
  'member',
  'legislative',
  'post'],
 'gouryella>>>NN': ['musician', 'production', 'team'],
 'diamicton>>>NN': ['dry', 'land', 'particle', 'clay'],
 'teguise (municipality)>>>NN': ['island'],
 'teguise (village)>>>VBP': ['island'],
 'rock-o-rama>>>NN': [

In [76]:
np.save("sememe_network_dict_en_wiki_2000",sememe_network)
np.save("sememe_network_cdv_en_wiki_2000",vocab)

In [1]:
sememe_network_dict_en_wiki_2000

NameError: name 'sememe_network_dict_en_wiki_2000' is not defined